In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [42]:
# site data
URL = "https://kreyol.com/dictionary/"
pages = ["Aa.html", "Bb.html", "Cc.html", "Dd.html", "Ee.html", 
        "Ff.html", "Gg.html", "Hh.html", "Ii.html", "Jj.html", 
        "Kk.html","Ll.html", "Mm.html", "Nn.html", "Oo.html",
        "Pp.html", "Qq.html", "Rr.html", "Ss.html", "Tt.html",
        "Uu.html", "Vv.html", "Ww.html", "Xx.html", "Yy.html",
        "Zz.html"]

In [41]:
# x = URL + pages[0]
# print(x)
# page = requests.get(URL + "Tt.html")
# # print(page.text)

# soup = BeautifulSoup(page.content, "html.parser")

# y = soup.find_all("p")
# # print(y)
# words = []
# count = 0
# for element in y:
#     # print(type(element))
#     # print(type(element.text))
#     if "/" in element.text and "href" not in element.text:
# #         print(count, "-", element)
#         words.append(element.text)
#     count += 1
    
# print(len(words))
# print(words)



In [30]:
# a = words[0].split("\n")
# # print(a[0] == "")
# # print(a)

# wordlist = pd.DataFrame(columns=["word", "creole_word", "creole name"])
# flip = False

# for entry in a:
#     if entry != "":  # check if the itme isn't an empty string
#         if not flip and "Kreyol / English" in entry: # after this page on the site the side the english is on and the side the creole is on flips
#             flip = True
#             continue
#         if "/" in entry:
#             text = ["", "", "hatian"]
#             if not flip:
#                 temp = entry.split("/")
# #                 print(temp)
#                 text[0] = temp[0].strip()
#                 text[1] = temp[1].strip()
#             else:
#                 temp = entry.split("/")
# #                 print(temp)
#                 text[0] = temp[1].strip()
#                 text[1] = temp[0].strip()

#             wordlist.loc[len(wordlist.index)] = text

In [44]:
wordlist = pd.DataFrame(columns=["word", "creole_word", "creole name"])

for site_page in pages: # loop through all the alphabet pages 
    x = URL + site_page
    print(x)
    
    page = requests.get(x)
    

    soup = BeautifulSoup(page.content, "html.parser") # pull the data from the site
    y = soup.find_all("p")

    words = []
    count = 0
    for element in y: # go through all the elements soup got and only save those elements that have a slash - the reason that all definitions have slashes in them 
        if "/" in element.text and "href" not in element.text:
            words.append(element.text)
        count += 1


    flip = False
    words_split = []
    for a in words:
        z = a.split("\n") # split the huge string grabbed by the 
        words_split.extend(z)

    for entry in z: # loop through every entry that was found
        if entry != "":  # check if the itme isn't an empty string
            if not flip and "Kreyol / English" in entry: # after this page on the site the side the english is on and the side the creole is on flips
                flip = True
                continue
            if "/" in entry:
                text = ["", "", "hatian"]
                if not flip:
                    temp = entry.split("/")
                    # print(temp)
                    text[0] = temp[0].strip()
                    text[1] = temp[1].strip()
                else:
                    temp = entry.split("/")
                    # print(temp)
                    text[0] = temp[1].strip()
                    text[1] = temp[0].strip()

                wordlist.loc[len(wordlist.index)] = text


https://kreyol.com/dictionary/Aa.html
https://kreyol.com/dictionary/Bb.html
https://kreyol.com/dictionary/Cc.html
https://kreyol.com/dictionary/Dd.html
https://kreyol.com/dictionary/Ee.html
https://kreyol.com/dictionary/Ff.html
https://kreyol.com/dictionary/Gg.html
https://kreyol.com/dictionary/Hh.html
https://kreyol.com/dictionary/Ii.html
https://kreyol.com/dictionary/Jj.html
https://kreyol.com/dictionary/Kk.html
https://kreyol.com/dictionary/Ll.html
https://kreyol.com/dictionary/Mm.html
https://kreyol.com/dictionary/Nn.html
https://kreyol.com/dictionary/Oo.html
https://kreyol.com/dictionary/Pp.html
https://kreyol.com/dictionary/Qq.html
https://kreyol.com/dictionary/Rr.html
https://kreyol.com/dictionary/Ss.html
https://kreyol.com/dictionary/Tt.html
https://kreyol.com/dictionary/Uu.html
https://kreyol.com/dictionary/Vv.html
https://kreyol.com/dictionary/Ww.html
https://kreyol.com/dictionary/Xx.html
https://kreyol.com/dictionary/Yy.html
https://kreyol.com/dictionary/Zz.html


In [47]:
wordlist

,word,creole_word,creole name
0,"a, an",youn,hatian
1,aback,pa sipriz (taken by surprise),hatian
2,abandon,"abandone, kite",hatian
3,abase,"abese (to lower in rank-office-prestige), ~dim...",hatian
4,abash,demoralize (to destroy the self-confidence of),hatian
...,...,...,...
5817,zouti,tool,hatian
5818,zuit (zwit),oyster,hatian
5819,zwa,goose,hatian
5820,zwazo,bird,hatian


In [48]:
wordlist.to_csv("hatian_creole_dictionary.csv")